# Automatizando com TPOT

O TPOT foi criado para ser um assistente que oferece ideias sobre como resolver um problema específico de aprendizado de máquina explorando configurações de pipeline que você pode nunca ter considerado e, em seguida, deixa o ajuste fino para técnicas de ajuste de parâmetro mais restritas, como grid search. Embora tenha capacidades básicas de feature engineering, elas são bastante incipientes ainda. Mais [aqui](http://epistasislab.github.io/tpot/)

In [4]:
#%%capture
#!pip install tpot

In [5]:
import pandas as pd
from tpot import TPOTRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [6]:
#from google.colab import drive
#drive.mount('/content/drive')

In [7]:

#data = pd.read_csv('drive/MyDrive/merge_iex_censo_banco2015-2021.csv')
data = pd.read_csv("/home/freitas/Downloads/internet2/data/merge_iex_censo_banco2015-2021.csv")
#data = pd.read_csv('drive/MyDrive/merge_iex_censo_banco.csv')
data.drop(columns=['Unnamed: 0'],inplace=True)
data.drop_duplicates(inplace=True)


In [8]:
data

,CNPJ,ANO,IEX,CO_ESCOLA,SG_UF,CO_MUNICIPIO,TP_SITUACAO_FUNCIONAMENTO,TP_DEPENDENCIA,TP_LOCALIZACAO,IN_LOCAL_FUNC_GALPAO,...,IN_SALA_DIRETORIA,IN_SECRETARIA,IN_COMPUTADOR,IN_EQUIP_COPIADORA,IN_EQUIP_IMPRESSORA,IN_INTERNET,IN_REGULAR,IN_EJA,possui_agencia_BB,POP
0,4033254000167,2015,2.957329,12004898,AC,1200302,1,2,2,0,...,0,0,0,0,0,0,1,0,1,32385
1,4033254000167,2015,2.957329,12022373,AC,1200302,1,2,2,0,...,0,0,0,0,0,0,1,0,1,32385
2,4033254000167,2015,2.957329,12031356,AC,1200336,1,2,2,1,...,0,0,0,0,0,0,1,0,1,17173
3,4033254000167,2015,2.957329,12027480,AC,1200500,1,2,2,0,...,0,0,0,0,0,0,1,0,1,41750
4,4033254000167,2015,2.957329,12028819,AC,1200401,1,2,2,0,...,0,0,0,0,0,0,1,0,1,370550
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
854610,18382268000109,2021,9.785132,43004040,RS,4322400,1,3,1,0,...,1,1,1,1,1,1,1,0,1,126866
854611,18382330000154,2021,9.889496,43004555,RS,4302154,1,3,1,0,...,1,0,1,0,0,1,1,0,0,2092
854612,17964207000189,2021,9.939361,43201954,RS,4318481,1,3,1,0,...,1,1,1,1,1,1,1,0,0,4865
854613,21249243000138,2021,10.000000,43314074,RS,4314423,1,3,2,0,...,1,1,1,0,0,1,1,0,0,5738


In [9]:
data = data.drop(columns=['CNPJ','CO_ESCOLA','CO_MUNICIPIO','TP_SITUACAO_FUNCIONAMENTO','TP_DEPENDENCIA','TP_LOCALIZACAO'])

In [10]:
data['SG_UF'] = data['SG_UF'].astype('category')
#data['CO_MUNICIPIO'] = data['CO_MUNICIPIO'].astype('category')
data_train = data.query('ANO < 2021')
data_test = data.query('ANO == 2021')
data_train.drop(columns=['ANO'],inplace=True)
data_test.drop(columns=['ANO'],inplace=True)


/tmp/ipykernel_39135/2792222690.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train.drop(columns=['ANO'],inplace=True)
/tmp/ipykernel_39135/2792222690.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test.drop(columns=['ANO'],inplace=True)


In [11]:
X_train = data_train.drop(columns=['IEX'])
y_train = data_train['IEX']
X_test = data_test.drop(columns=['IEX'])
y_test = data_test['IEX']



## Pre-processing

In [12]:
!pip install feature_engine

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.0/290.0 kB 229.2 kB/s eta 0:00:00m eta 0:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 4.3 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.8/233.8 kB 2.3 MB/s eta 0:00:005.5 MB/s eta 0:00:01


In [13]:
# for one hot encoding with feature-engine
from feature_engine.encoding import OneHotEncoder

# let's create the encoder

ohe_enc = OneHotEncoder(
    top_categories=None,
    drop_last=False)  # to return k-1, false to return k

In [ ]:
# fit the encoder to the train set: it will learn the variables and 
# categories to encode

ohe_enc.fit(X_train)

OneHotEncoder()

In [ ]:
# we can see which variables the encoder will encode

ohe_enc.variables

In [ ]:
# let's transform train and test set

X_train = ohe_enc.transform(X_train)
X_test = ohe_enc.transform(X_test)

/usr/local/lib/python3.7/dist-packages/feature_engine/encoding/one_hot.py:253: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  X[feature] == category, 1, 0


In [ ]:
# let's inspect the encoded train set

X_train

In [ ]:
X_train.shape

In [ ]:
# set up the scaler
scaler = MinMaxScaler()

# fit the scaler to the train set, it will learn the parameters
scaler.fit(X_train[['POP']])

# transform train and test sets
X_train['POP'] = scaler.transform(X_train[['POP']])
X_test['POP'] = scaler.transform(X_test[['POP']])

# CatBoost

# Model

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator
from tpot.export_utils import set_param_recursive

# Average CV score on the training set was: -13.355637624157364
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=SGDRegressor(alpha=0.01, eta0=1.0, fit_intercept=False, l1_ratio=0.0, learning_rate="invscaling", loss="squared_error", penalty="elasticnet", power_t=0.1)),
    RandomForestRegressor(bootstrap=True, max_features=0.4, min_samples_leaf=9, min_samples_split=9, n_estimators=100)
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 42)

exported_pipeline.fit(X_train,y_train)
results = exported_pipeline.predict(X_test)

In [ ]:
from sklearn.model_selection import cross_validate
result = cross_validate(exported_pipeline, X_test, y_test, 
                         cv=5, scoring=['neg_mean_squared_error','neg_mean_absolute_percentage_error','neg_mean_absolute_error'])
print(result['test_neg_mean_squared_error'].mean())
print(result['test_neg_mean_absolute_error'].mean())


In [ ]:
y_mean = np.mean(y_train)
print(f'The mean prediction is {y_mean:.3f}')

absolute_error_test_baseline = [np.abs(y_true - y_mean) for y_true in y_test]
mean_absolute_error_test_baseline = np.mean(absolute_error_test_baseline)

print(f'The benchmark mean absolute error is of {mean_absolute_error_test_baseline:.3f}')

In [ ]:
y_pred = exported_pipeline.predict(X_test)


In [ ]:
absolute_error_test_model = np.abs(y_test - y_pred)
mean_absolute_error_test_model = np.mean(absolute_error_test_model)

print(f'Model mean absolute error is of {mean_absolute_error_test_model:.3f}')

In [ ]:
absolute_error_test_model


In [ ]:
from sklearn.metrics import r2_score
score = r2_score(y_test,y_pred)
score

In [ ]:
exported_pipeline.score(X_test,y_test)

In [ ]:
data_test['IEX_pred'] = y_pred
data_test

In [ ]:
aoeuaou

In [ ]:
#X_test['y_pred'] = y_pred
#data_final = X_test

In [ ]:
# y_mean = np.
# baseline_score = r2_score(y_test,)
# baseline_score

In [ ]:
import sklearn
sorted(sklearn.metrics.SCORERS.keys()) 

In [ ]:
data = pd.read_csv('drive/MyDrive/merge_iex_censo_banco.csv')
data = data.drop(columns=['Unnamed: 0','CNPJ','CO_ESCOLA','CO_MUNICIPIO','ANO'])


In [ ]:

bin_labels_IEX = ['pessimo', 'ruim', 'bom', 'muito_bom']
df['IEX'] = data.qcut(df['IEX'], q=4,
                       labels = bin_labels_IEX)


In [ ]:
from sklearn.model_selection import cross_validate
result = cross_validate(exported_pipeline, features, tpot_data['IEX'], 
                         cv=5, scoring=['neg_mean_squared_error','neg_mean_absolute_percentage_error','neg_mean_absolute_error'])
print(result['test_neg_mean_squared_error'].mean())
print(result['test_neg_mean_absolute_error'].mean())


In [ ]:
%%capture
!pip install shap

In [ ]:
import shap

In [ ]:
%%capture
# criando um explainer
explainer = shap.KernelExplainer(tpot.predict,X_test)

# Definindo os shap values pelo explainer
shap_values = explainer.shap_values(X_test)
# Definindo um objeto explainer para uso com alguns gráficos
# shap_exp = explainer(X_test)
# Valor esperado / base / de referência = o valor que seria previsto se não soubéssemos quaisquer características da saída atual ”
print('Valor base:', explainer.expected_value)

pd.DataFrame(shap_values).head()

In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value, 
                shap_values[0,:], X_test.iloc[0,:])

In [ ]:
shap.initjs()
shap.summary_plot(shap_values, 
                  X_test, plot_type="bar")

In [ ]:
shap.initjs()
shap.summary_plot(shap_values, X_test)

In [ ]:
shap.initjs()
shap.dependence_plot(ind='pclass', interaction_index='age',
                     shap_values=shap_values, 
                     features=X_test)

## Usando outra abordagem: catboost

In [ ]:
!pip install catboost

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split

import catboost
print('catboost version:', catboost.__version__)
from catboost import CatBoostClassifier 
titanic_df = titanic

In [ ]:
# map categorical features
titanic_catboost_ready_df = titanic_df.dropna() 

features = [feat for feat in list(titanic_catboost_ready_df) 
            if feat != 'survived']
print(features)
categorical_features = np.where(titanic_catboost_ready_df[features].dtypes != np.float)[0]
 

X_train, X_test, y_train, y_test = train_test_split(titanic_df[features], 
                                                    titanic_df[['survived']], 
                                                    test_size=0.3, 
                                                     random_state=1)
 
params = {'iterations':5000,
        'learning_rate':0.01,
        'cat_features':categorical_features,
        'depth':3,
        'eval_metric':'AUC',
        'verbose':200,
        'od_type':"Iter", 
        'od_wait':500, 
        'random_seed': 1
          }

cat_model = CatBoostClassifier(**params)
cat_model.fit(X_train, y_train,   
          eval_set=(X_test, y_test), 
          use_best_model=True, 
          plot=True  
         );
 

In [ ]:
import shap 
from catboost import CatBoostClassifier, Pool
shap_values = cat_model.get_feature_importance(Pool(X_test, label=y_test,cat_features=categorical_features) ,
                                               type="ShapValues")
 
expected_value = shap_values[0,-1]
shap_values = shap_values[:,:-1]

shap.initjs()
shap.force_plot(expected_value, shap_values[0,:], X_test.iloc[0,:])

In [ ]:
shap.initjs()
shap.force_plot(expected_value, shap_values[1,:], X_test.iloc[1,:])